In [19]:
# Necessary Imports
import pandas as pd
import tensorflow as tf
import nltk
import sklearn
import re
import numpy as np

In [4]:
# Initial Data Preprocessing for Fake Dataset
fake_dataset=pd.read_csv("Fake.csv")
fake_dataset=fake_dataset.drop(columns=["date","subject"])
fake_dataset['authenticity']=0
fake_dataset.head()

,title,text,authenticity
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0


In [5]:
# Initial Data Preprocessing for True Dataset
true_dataset=pd.read_csv("True.csv")
true_dataset=true_dataset.drop(columns=["date","subject"])
true_dataset['authenticity']=1
true_dataset.head()

,title,text,authenticity
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [6]:
# Checking for Null Values if any
print(fake_dataset.isna().sum(),"\n")
print(true_dataset.isna().sum())

title           0
text            0
authenticity    0
dtype: int64 

title           0
text            0
authenticity    0
dtype: int64


In [7]:
# Combining both datafarmes in to a single dataframe
dataset=pd.concat([true_dataset,fake_dataset])
dataset=dataset.sample(frac=1)
dataset=dataset.reset_index()
dataset=dataset.drop(columns=["index"])
dataset

,title,text,authenticity
0,"'Study, pray, vote,' Pope tells Americans ahea...",ABOARD THE PAPAL PLANE (Reuters) - Pope Franci...,1
1,U.S. lawmakers reach deal for Senate vote on R...,WASHINGTON (Reuters) - U.S. lawmakers reached ...,1
2,U.S. conservative leader optimistic of common ...,WASHINGTON (Reuters) - Republicans in the U.S....,1
3,WATCH HILLARY CLINTON Go Full On Socialist…You...,The Death Tax is the hard earned money you hav...,0
4,"EP #13: Patrick Henningsen LIVE – ‘Fake News, ...",In response to the establishment media s con...,0
...,...,...,...
44893,The Shocking Detail Trump Doesn’t Want You To...,A rather interesting detail has emerged about ...,0
44894,BRITISH MP Nigel Evans SHAMES Anti-Trump Parli...,During a meeting of the Welsh members of the P...,0
44895,"Factbox: Trump to meet with Oklahoma governor,...",(Reuters) - U.S. Republican President-elect Do...,1
44896,U.S. House Democrats' Pelosi: administration m...,WASHINGTON (Reuters) - U.S. House Democratic L...,1


In [8]:
X=dataset.drop('authenticity',axis=1)
Y=dataset['authenticity']

In [9]:
# Embedding layer
from tensorflow.keras.layers import Embedding
# Add padding so as to make the sentences equal in length
from tensorflow.keras.preprocessing.sequence import pad_sequences
# For creating a sequential Model
from tensorflow.keras.models import Sequential
# Necessary import for One-hot encoding
from tensorflow.keras.preprocessing.text import one_hot
# Long short-term memory neural network
from tensorflow.keras.layers import LSTM
# Dense layer-> Final layer must have one node showing probalities
from tensorflow.keras.layers import Dense

In [10]:
# Vocabulary Size
voc_size=5000

In [11]:
# Final Data Preprocessing
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Daniel
[nltk_data]     Saldanha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# Stemming is the process of reducing a word to it's root word
# Stopwords are words that do not add much meaning to a sentance
data=X.copy()
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(data)):
    # To show running of the process
    print("*",end='')
    temp=re.sub('[^a-zA-Z]',' ',data['title'][i])
    temp=temp.lower()
    temp=temp.split()
    temp=[ps.stem(word) for word in temp if not word in stopwords.words('english')]
    temp=' '.join(temp)
    corpus.append(temp)

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [15]:
# One-hot Representation
onehot_rep=[one_hot(words,voc_size)for words in corpus]
onehot_rep

[[3367, 4690, 1136, 194, 2025, 3020, 872, 4314],
 [4906, 4253, 110, 1780, 2159, 1136, 3431, 2486],
 [4906, 1699, 2920, 940, 1964, 1008, 2244],
 [28, 60, 721, 334, 2929, 531, 2943, 441, 2225, 2321],
 [736, 2492, 4702, 1803, 203, 4903, 203, 599, 2367, 1907, 3879],
 [346, 1338, 1079, 4882, 3074, 3077, 2699, 4089],
 [204, 3529, 2311, 460],
 [4906, 3124, 1021, 857, 1914, 1332, 671, 2388, 700],
 [2176, 3751, 1830, 1079, 188, 3488, 4089, 1037],
 [2504, 4178, 2144, 3567, 1290, 1550, 1296],
 [1844, 952, 1913, 3585, 2921, 2227, 1481, 721, 2321],
 [3438,
  2557,
  3009,
  1302,
  3054,
  3969,
  837,
  576,
  3425,
  2839,
  86,
  3398,
  3275,
  3969,
  624,
  1831,
  2321,
  1063,
  3425,
  1937,
  783,
  387],
 [4906, 1914, 2447, 1250, 3406, 4296, 3529, 4984, 4785],
 [296, 1572, 3529, 2548, 304, 2122, 3647, 1106, 2967, 358, 2321],
 [2037, 3026, 1028, 1978, 1916, 1699, 4352, 1825, 2321],
 [3470, 343, 2295, 4226, 3305, 4363, 195, 1562],
 [2236, 4431, 736, 1484],
 [721, 1009, 4785, 4778, 3458, 89

In [16]:
# Embedding Representation
sent_length=20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3020  872 4314]
 [   0    0    0 ... 1136 3431 2486]
 [   0    0    0 ... 1964 1008 2244]
 ...
 [   0    0    0 ... 3368  883 1226]
 [   0    0    0 ... 3297 2737 4123]
 [   0    0    0 ... 2789   31 2311]]


In [17]:
# Creating the Model
from tensorflow.keras.layers import Dropout
embedded_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
X_final=np.array(embedded_docs)
y_final=np.array(Y)

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [22]:
# Loading Previous Saved Weights
model.load_weights('saved_weights/')

In [23]:
# Model Training
#model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)
model.fit(X_train,y_train,validation_split=0.3,epochs=10,batch_size=64)

Epoch 1/10
329/329 [==============================] - 4s 11ms/step - loss: 0.4477 - accuracy: 0.8405 - val_loss: 0.2055 - val_accuracy: 0.9155
Epoch 2/10
329/329 [==============================] - 3s 10ms/step - loss: 0.1306 - accuracy: 0.9485 - val_loss: 0.1678 - val_accuracy: 0.9357
Epoch 3/10
329/329 [==============================] - 3s 10ms/step - loss: 0.0734 - accuracy: 0.9735 - val_loss: 0.1832 - val_accuracy: 0.9346
Epoch 4/10
329/329 [==============================] - 3s 10ms/step - loss: 0.0431 - accuracy: 0.9857 - val_loss: 0.2087 - val_accuracy: 0.9355
Epoch 5/10
329/329 [==============================] - 3s 10ms/step - loss: 0.0245 - accuracy: 0.9920 - val_loss: 0.2540 - val_accuracy: 0.9310
Epoch 6/10
329/329 [==============================] - 3s 10ms/step - loss: 0.0148 - accuracy: 0.9960 - val_loss: 0.2856 - val_accuracy: 0.9340
Epoch 7/10
329/329 [==============================] - 3s 10ms/step - loss: 0.0092 - accuracy: 0.9974 - val_loss: 0.3486 - val_accuracy: 0.9292

In [24]:
# Saving the Model weights
model.save_weights('saved_weights/')

In [25]:
# Performance Metrics and Accuracy
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[7341,  533],
       [ 454, 6489]], dtype=int64)

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.933387325369508

In [28]:
# Training the model using the test data aswell
#Loading Previous Saved Wights
model.load_weights('saved_weights/')
model.fit(X_test,y_test,validation_split=0.3,epochs=10,batch_size=64)

Epoch 1/10
163/163 [==============================] - 2s 13ms/step - loss: 0.2125 - accuracy: 0.9371 - val_loss: 0.1541 - val_accuracy: 0.9366
Epoch 2/10
163/163 [==============================] - 2s 12ms/step - loss: 0.0776 - accuracy: 0.9765 - val_loss: 0.1547 - val_accuracy: 0.9442
Epoch 3/10
163/163 [==============================] - 2s 12ms/step - loss: 0.0459 - accuracy: 0.9858 - val_loss: 0.1701 - val_accuracy: 0.9460
Epoch 4/10
163/163 [==============================] - 2s 12ms/step - loss: 0.0253 - accuracy: 0.9936 - val_loss: 0.1836 - val_accuracy: 0.9478
Epoch 5/10
163/163 [==============================] - 2s 12ms/step - loss: 0.0135 - accuracy: 0.9964 - val_loss: 0.2062 - val_accuracy: 0.9438
Epoch 6/10
163/163 [==============================] - 2s 12ms/step - loss: 0.0075 - accuracy: 0.9988 - val_loss: 0.2283 - val_accuracy: 0.9480
Epoch 7/10
163/163 [==============================] - 2s 12ms/step - loss: 0.0039 - accuracy: 0.9991 - val_loss: 0.2559 - val_accuracy: 0.9438

In [52]:
# Result for a single input

# Enter News Title below
sample_data="Is covid fake"

# Data Preprocessing
sample_data=re.sub('[^a-zA-Z]',' ',sample_data)
sample_data=sample_data.lower()
sample_data=sample_data.split()
sample_data=[ps.stem(word) for word in sample_data if not word in stopwords.words('english')]
sample_data=' '.join(sample_data)

# One-hot Encoding
onehot_rep=[one_hot(sample_data,voc_size)]
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
sample_data=np.array(embedded_docs)

In [53]:
pred=model.predict_classes(sample_data)
if(pred[0][0] ==0):
    print("The Title is Fake")
else:
    print("The Title is Real")

The Title is Fake
